# Workshop 3: Cleaning, Manipulating, and Exploring Data with Pandas
[Placeholder for intro materials]

Notes:

- Take out combining datasets?
- No renaming columns?
- write an example for dropna()
- Add in an apply() function

In [3]:
# Import the Pandas library as pd (callable in our code as pd)
import pandas as pd

In [4]:
# Import the dataset as a DataFrame

# The file location
csv_file_url = 'https://raw.githubusercontent.com/NCSU-Libraries/data-viz-instruction/main/MI_REU_2021/data/perovskite_DFT_EaH_FormE.csv'

# Read in the file and print out the DataFrame
ts_csv = pd.read_csv(csv_file_url)
ts_csv.head()

,Material Composition,A site #1,A site #2,A site #3,B site #1,B site #2,B site #3,X site,Empty,Empty.1,Empty.2,Empty.3,Empty.4,energy_above_hull (meV/atom),formation_energy (eV/atom)
0,Ba1Sr7V8O24,Ba,Sr,NaN,V,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,29.747707,-2.113335
1,Ba2Bi2Pr4Co8O24,Ba,Bi,Pr,Co,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,106.702335,-1.311863
2,Ba2Ca6Fe8O24,Ba,Ca,NaN,Fe,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,171.608093,-1.435607
3,Ba2Cd2Pr4Ni8O24,Ba,Cd,Pr,Ni,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,284.898190,-0.868639
4,Ba2Dy6Fe8O24,Ba,Dy,NaN,Fe,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,270.007913,-1.746806


## Combining datasets

## Removing data
### Drop columns
We can reduce the size of our dataset by removing unnecessary columns of data using the DataFrame `drop()` method.

In [41]:
# Remove the empty columns using "drop()""
ts_csv_clean = ts_csv.drop(columns=["Empty", "Empty.1", "Empty.2", "Empty.3", "Empty.4"])

# Print out the first five records of the DataFrame
ts_csv_clean.head()

,Material Composition,A site #1,A site #2,A site #3,B site #1,B site #2,B site #3,X site,energy_above_hull (meV/atom),formation_energy (eV/atom)
0,Ba1Sr7V8O24,Ba,Sr,NaN,V,NaN,NaN,O,29.747707,-2.113335
1,Ba2Bi2Pr4Co8O24,Ba,Bi,Pr,Co,NaN,NaN,O,106.702335,-1.311863
2,Ba2Ca6Fe8O24,Ba,Ca,NaN,Fe,NaN,NaN,O,171.608093,-1.435607
3,Ba2Cd2Pr4Ni8O24,Ba,Cd,Pr,Ni,NaN,NaN,O,284.898190,-0.868639
4,Ba2Dy6Fe8O24,Ba,Dy,NaN,Fe,NaN,NaN,O,270.007913,-1.746806


### Remove NaN

In [92]:
print(ts_csv_clean.shape)

ts_csv_clean.dropna(how='all')
ts_csv_clean.shape

(1929, 12)


(1929, 12)

## Calculating new data
### Calculating with Expressions
We may want to add a new column that is calculated based on other columns. In this example, we create a new column (`Number of elements`) that tells us the number of elements over all of the sites.

We will create a list of all of the columns that we want to include, then create a new column called `Number of elements` that uses `.count()` to count the number of values included in those columns.

In [80]:
# Create a new column for the number of elements found
elements_columns = ['A site #1', 'A site #2','A site #3', 'B site #1', 'B site #2', 'B site #3', 'X site']
ts_csv_clean['Number of elements'] = ts_csv_clean[elements_columns].count(axis='columns')

# Print out the new column
ts_csv_clean[['A site #1', 'A site #2','A site #3', 'B site #1', 'B site #2', 'B site #3', 'X site', 'Number of elements']]

,A site #1,A site #2,A site #3,B site #1,B site #2,B site #3,X site,Number of elements
0,Ba,Sr,NaN,V,NaN,NaN,O,4
1,Ba,Bi,Pr,Co,NaN,NaN,O,5
2,Ba,Ca,NaN,Fe,NaN,NaN,O,4
3,Ba,Cd,Pr,Ni,NaN,NaN,O,5
4,Ba,Dy,NaN,Fe,NaN,NaN,O,4
...,...,...,...,...,...,...,...,...
1924,Y,NaN,NaN,V,Co,NaN,O,4
1925,Y,NaN,NaN,V,Cr,NaN,O,4
1926,Y,NaN,NaN,V,Fe,NaN,O,4
1927,Y,NaN,NaN,V,Ni,NaN,O,4


### Calculating with apply functions
Sometimes you need to create a new column based on more complex manipulation of existing data. In this example, we use the `apply()` method to apply a function along the rows in the column... [fill in with example]

In [76]:
def 

SyntaxError: invalid syntax (<ipython-input-76-455680ca2399>, line 1)

## Replacing data
We can replace values in a column based on conditions, similar to "find and replace." In this example, we change two element abbreviations to their full names in the `A site #2` column.

In [90]:
# Replace 'Sr' and 'Ca' abbreviations with "Strontium" and "Calcium"
ts_csv_clean['A site #2'].replace({'Sr': 'Strontium', 'Ca': 'Calcium'}, inplace=True)

# Print out the updated column of data
ts_csv_clean['A site #2'].head()

0    Strontium
1           Br
2      Calcium
3           Cd
4           Dy
Name: A site #2, dtype: object

In [91]:
#Change the element names back to their abbreviations
ts_csv_clean['A site #2'].replace({'Strontium': 'Sr', 'Calcium': 'Ca'}, inplace=True)

ts_csv_clean['A site #2'].head()

0    Sr
1    Br
2    Ca
3    Cd
4    Dy
Name: A site #2, dtype: object

## Filtering data
### Conditional filtering
We can filter our data using conditional statements. This can help us remove unecessary rows of data or observe a specific range of data.

In [16]:
# Filter the data to only see perovskite oxides where Calcium was found at A site #1
calcium = ts_csv_clean[ts_csv_clean['A site #1'] == 'Ca']

# Print out the filtered data
calcium

,Material Composition,A site #1,A site #2,A site #3,B site #1,B site #2,B site #3,X site,energy_above_hull (meV/atom),formation_energy (eV/atom),Number of elements
485,Ca2La6Co8O24,Ca,La,NaN,Co,NaN,NaN,O,50.025452,-1.693684,placeholder
486,Ca2La6Cr8O24,Ca,La,NaN,Cr,NaN,NaN,O,0.000000,-2.297925,placeholder
487,Ca2La6Fe8O24,Ca,La,NaN,Fe,NaN,NaN,O,0.000000,-2.034384,placeholder
488,Ca2La6Ga8O24,Ca,La,NaN,Ga,NaN,NaN,O,73.974147,-2.173541,placeholder
489,Ca2La6Mn8O24,Ca,La,NaN,Mn,NaN,NaN,O,32.789931,-2.086983,placeholder
...,...,...,...,...,...,...,...,...,...,...,...
616,Ca8Ni6Co2O24,Ca,NaN,NaN,Ni,Co,NaN,O,361.579777,-0.866839,placeholder
617,Ca8Ni7Co1O24,Ca,NaN,NaN,Ni,Co,NaN,O,378.453371,-0.824937,placeholder
618,Ca8Ni8O24,Ca,NaN,NaN,Ni,NaN,NaN,O,397.239965,-0.781123,placeholder
619,Ca8Ti8O24,Ca,NaN,NaN,Ti,NaN,NaN,O,12.448077,-2.774067,placeholder


In [26]:
# Filter the data to only see only perovskite oxides with a formation energy greater than -1
high_fe = ts_csv_clean[ts_csv_clean['formation_energy (eV/atom)'] > -1]

# Print out the filtered data
high_fe

,Material Composition,A site #1,A site #2,A site #3,B site #1,B site #2,B site #3,X site,energy_above_hull (meV/atom),formation_energy (eV/atom),Number of elements
3,Ba2Cd2Pr4Ni8O24,Ba,Cd,Pr,Ni,NaN,NaN,O,284.898190,-0.868639,placeholder
41,Ba2Sr6Ni8O24,Ba,Sr,NaN,Ni,NaN,NaN,O,258.828051,-0.801693,placeholder
54,Ba3Cd1Pr4Ni8O24,Ba,Cd,Pr,Ni,NaN,NaN,O,257.075963,-0.967393,placeholder
59,Ba4Ba4Mn2Fe6O24,Ba,Ba,NaN,Mn,Fe,NaN,O,271.906598,-0.931270,placeholder
62,Ba4Ca4Co4Ni4O24,Ba,Ca,NaN,Co,Ni,NaN,O,296.095913,-0.941304,placeholder
...,...,...,...,...,...,...,...,...,...,...,...
1561,Sr8Fe1Ni7O24,Sr,NaN,NaN,Fe,Ni,NaN,O,243.428371,-0.908675,placeholder
1597,Sr8Ni6Co2O24,Sr,NaN,NaN,Ni,Co,NaN,O,241.664953,-0.906028,placeholder
1598,Sr8Ni7Co1O24,Sr,NaN,NaN,Ni,Co,NaN,O,252.767162,-0.864348,placeholder
1599,Sr8Ni8O24,Sr,NaN,NaN,Ni,NaN,NaN,O,263.543371,-0.822993,placeholder


In [27]:
# Filter the data to only see perovskite oxides where Calcium was found at A site #1 
# that also have a formation energy greater than -1
calcium_fe = ts_csv_clean[(ts_csv_clean['A site #1'] == 'Ca') & (ts_csv_clean['formation_energy (eV/atom)'] > -1)]

# Print out the filtered data
calcium_fe

,Material Composition,A site #1,A site #2,A site #3,B site #1,B site #2,B site #3,X site,energy_above_hull (meV/atom),formation_energy (eV/atom),Number of elements
512,Ca4Ba4Co4Ni4O24,Ca,Ba,NaN,Co,Ni,NaN,O,292.390663,-0.945009,placeholder
516,Ca4Ba4Fe1Ni7O24,Ca,Ba,NaN,Fe,Ni,NaN,O,323.822637,-0.826354,placeholder
518,Ca4Ba4Fe2Ni6O24,Ca,Ba,NaN,Fe,Ni,NaN,O,309.856621,-0.910077,placeholder
520,Ca4Ba4Mn1Ni7O24,Ca,Ba,NaN,Mn,Ni,NaN,O,340.437655,-0.833147,placeholder
522,Ca4Ba4Mn2Fe6O24,Ca,Ba,NaN,Mn,Fe,NaN,O,956.831956,-0.666962,placeholder
523,Ca4Ba4Mn2Ni6O24,Ca,Ba,NaN,Mn,Ni,NaN,O,318.399908,-0.948350,placeholder
532,Ca4Ba4Ni4Co4O24,Ca,Ba,NaN,Ni,Co,NaN,O,294.310413,-0.943089,placeholder
533,Ca4Ba4Ni6Co2O24,Ca,Ba,NaN,Ni,Co,NaN,O,357.145351,-0.830199,placeholder
534,Ca4Ba4Ni7Co1O24,Ca,Ba,NaN,Ni,Co,NaN,O,351.817502,-0.782065,placeholder
557,Ca4Sr4Fe1Ni7O24,Ca,Sr,NaN,Fe,Ni,NaN,O,313.927543,-0.884090,placeholder


## Aggregating data
### Unique
It may be interesting to see the total number of unique elements that are recorded in our dataset at a certain site. We can use the `unique()` method on the `B site #3` column to create an array of unique elements. 

The length of this array will provide the number of unique elements found at that site.

In [28]:
# Create a list of the unique elements sounds at B site #3 with unique()
B3_elements = ts_csv_clean['B site #3'].unique()

# Print out the unique species
B3_elements

array([nan, 'Hf', 'Co', 'Fe', 'Cu', 'Mo', 'Ni', 'W'], dtype=object)

In [29]:
# Get the length of the new array (How many unique elements are there?)
len(B3_elements)

8

### Value counts
Value counts show how many instances there are of each unique entry in a column. Here, we are interested in seeing how many of each type of element there are at A site #2.

We will specify the `A site #2` column in our dataset and call the method `value_counts()`. This will return a Series with an index label of each unique element and a value corresponding to the count of that element in the `A site #2` column.

In [62]:
# Count the occurance of unique values on the column 'A site #2'
A2_elements = ts_csv_clean['A site #2'].value_counts()

A2_elements

Sr    162
Zn    144
Ca    143
Y     132
Ba    117
Pr    104
La     74
Ho     61
Sm     45
Gd     44
Dy     43
Nd     42
Sn     14
Cd     14
Bi     11
Mg      7
Ce      3
Er      1
Name: A site #2, dtype: int64

### Minimum, maximum, average
We can also calculate aggregates like the minimum, maximum, and mean of values in a DataFrame or Series. Here are a few examples:

- `mean()` to find the average of a range
- `min()` to find the smallest value
- `max()` to find the largest value
- `sum()` to sum the values of a range

In [32]:
# Calculate the minimum values for each column
ts_csv_clean.min()

Material Composition            Ba1Sr7V8O24
A site #1                                Ba
B site #1                                Al
X site                                    O
energy_above_hull (meV/atom)              0
formation_energy (eV/atom)          -3.2085
Number of elements              placeholder
dtype: object

In [33]:
# Calculate the average height for energy above hull
ts_csv_clean['energy_above_hull (meV/atom)'].mean()

105.53263270409896

We can use the `agg()` method to call multiple aggregate functions at once.

In [34]:
# Calculate the minimum, maximum, and average hour for energy above hull
ts_csv_clean['energy_above_hull (meV/atom)'].agg(['min', 'max', 'mean'])

min       0.000000
max     956.831956
mean    105.532633
Name: energy_above_hull (meV/atom), dtype: float64

## Grouping data
We may be interested in seeing our data in groups. For example, what does the data look like if we group by the element found at A site #1? How many of each element are found?

We can do this by calling `groupby()` on our dataset and passing in the column we would like to group by. We will group our data by the column `A site #1`, find the count or `size()` of each column in the grouped data, then sort the data from smallest to largest.

In [61]:
# Group the dataset by "A site #1"
by_Asite1 = ts_csv_clean.groupby('A site #1')

# This creates a groupby object that contains information about the groups
print(type(by_Asite1))

# Find the size, or counts, of the grouped data
by_Asite1 = by_Asite1.size()

by_Asite1.sort_values()

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


A site #1
Cd      1
Ho      1
Mg      1
Sn      1
Bi      4
Dy     19
Gd     19
Nd     19
Sm     19
Ca    136
Y     258
Sr    302
Pr    330
La    338
Ba    481
dtype: int64

You can also use `groupby()` group data by multiple factors. Here we are grouping by `A site #1` and then `A site #2` to see the averages for different elements in those two sites.

In [60]:
# Group the data by A site #1 and then A site #2 and get the mean of the remaining columns using .mean()
ts_csv_clean.groupby(['A site #1', 'A site #2']).mean()

energy_above_hull (meV/atom)  formation_energy (eV/atom)
A site #1 A site #2                                                          
Ba        Ba                           271.906598                   -0.931270
          Bi                           105.932321                   -1.616462
          Ca                           239.308297                   -1.182568
          Cd                           142.473483                   -1.680978
          Ce                           108.880760                   -1.700416
...                                           ...                         ...
Y         Ba                           275.239004                   -1.395605
          Ca                           221.246470                   -1.484484
          Ho                            47.305564                   -2.348126
          Sr                           170.047628                   -1.515276
          Zn                           121.133217                   -1.864187

[69 rows x 2 columns]

## Visualization